# 200_prep_dataset1

## Purpose
One of the most important tasks in any data science project is the cleaning and verification of the raw data. In fact, many commentators will tell you that this makes up about 80% of the workload. Certainly, cleaning and validating the data can be a detailed and laborious process, and one that needs to be careful considered and documented. In this notebook we will get started, focusing mainly on dealing with null values, or missing values in our dataset. We will also group industries together. 

## Datasets
* _Input_: 100_dataset1.pkl
* _Output_: 200_dataset1.pkl



In [5]:
import os
import re
import sys
import hashlib
import pandas as pd
import numpy as np
%matplotlib inline
pd.set_option('display.max_columns', None)
module_path = os.path.abspath(os.path.join('../../data/..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Preparing First Dataset

In this notebook, our main aim is to prepare the first dataset for analysis. In our case this mainly involves removing null values, and grouping industries. If you remember from the 100_load_startup_datasets.ipynb, our first dataset will be used to analyse the first RQ: 
 * (RQ1: Correlation between a company’s industry and location, with the amount of funding they receive?)

We initially read in the dataset from the previous notebook and print out the first 5 rows. This gives a look into the enteries in our dataframe. Doing this enables us to make more informed decisions going forward with our analysis.

In [6]:
ds1_df = pd.read_pickle('../../data/processed/100_dataset1.pkl')
ds1_df.head(5)

,company_name,roles,country_code,state_code,region,city,status,category_list,category_group_list,funding_rounds,funding_total_usd,last_funding_on,founded_on,employee_count,org_uuid,primary_role,type
0,Intel,"company,investor",USA,CA,SF Bay Area,Santa Clara,ipo,"Hardware,Manufacturing,Product Design,Semicond...","Design,Hardware,Manufacturing,Science and Engi...",1,2510000.0,1968-07-31,1968-07-18,10000+,1e4f199c-363b-451b-a164-f94571075ee5,company,organization
1,Intercomp,company,USA,OH,Cleveland,Medina,operating,"Hardware,Software","Hardware,Software",1,549000.0,1970-12-31,1968-01-01,101-250,6681b1b0-0cea-6a4a-820d-60b15793fa66,company,organization
2,Gruppo Editoriale L'Espresso Spa,company,ITA,NaN,Rome,Rome,ipo,"News,Publishing","Content and Publishing,Media and Entertainment",1,NaN,1976-01-01,1955-01-01,1001-5000,e74463f7-94a8-ab7b-ba31-f4cd57a16570,company,organization
3,Microsoft,"company,investor",USA,WA,Seattle,Redmond,ipo,"Cloud Computing,Collaboration,Consumer Electro...","Consumer Electronics,Hardware,Internet Service...",1,1000000.0,1981-09-01,1975-04-04,10000+,fd80725f-53fc-7009-9878-aeecf1e9ffbb,company,organization
4,Compaq,"company,investor",USA,CA,SF Bay Area,Palo Alto,acquired,"Hardware,Information Technology,Software","Hardware,Information Technology,Software",1,1500000.0,1982-02-14,1982-02-14,11-50,10a3b2fd-b142-046b-7d8f-3b1aa4877aca,company,organization


It is important to know the size of the dataset you are working with. As you can see below, we are dealing with 638,270 rows and 17 columns. 

In [7]:
ds1_df.shape

(638270, 17)

Below we wanted to get an insight into the amount of categories of industry that we were dealing with, as well as the number of unique industries in our dataframe. As you can see below we ran into some trouble. The industries are grouped together and seperated with a comma, this means that if a company falls under multiple industries this is then counted as a single industry in our value_counts. For example "Commerce and Shopping,Financial Services,Food and Beverage,Software". This will cause problems with our analysis if we don't deal with this issue.

In [8]:
ds1_df['category_group_list'].value_counts()

Software                                                                                                                                                                                              18518
Health Care                                                                                                                                                                                           14710
Financial Services                                                                                                                                                                                    12861
Commerce and Shopping                                                                                                                                                                                  9548
Financial Services,Lending and Investments                                                                                                                                              

We then decided to remove any columns that provided a null value for the "funding_total_usd" column. This was quite an intuitive decision considering we are using funding as a measure of success in our analysis. We now only have 83,840 rows and 17 columns after removing the columns with no funding values. This is quite a drastic drop from our inital number of rows and columns, but fortunately we still have a huge amount of data to analysis. 

In [9]:
clean_df1 = ds1_df[~ds1_df["funding_total_usd"].isnull()]
clean_df1.shape

(83840, 17)

In [10]:
clean_df1.head(5)

,company_name,roles,country_code,state_code,region,city,status,category_list,category_group_list,funding_rounds,funding_total_usd,last_funding_on,founded_on,employee_count,org_uuid,primary_role,type
0,Intel,"company,investor",USA,CA,SF Bay Area,Santa Clara,ipo,"Hardware,Manufacturing,Product Design,Semicond...","Design,Hardware,Manufacturing,Science and Engi...",1,2510000.0,1968-07-31,1968-07-18,10000+,1e4f199c-363b-451b-a164-f94571075ee5,company,organization
1,Intercomp,company,USA,OH,Cleveland,Medina,operating,"Hardware,Software","Hardware,Software",1,549000.0,1970-12-31,1968-01-01,101-250,6681b1b0-0cea-6a4a-820d-60b15793fa66,company,organization
3,Microsoft,"company,investor",USA,WA,Seattle,Redmond,ipo,"Cloud Computing,Collaboration,Consumer Electro...","Consumer Electronics,Hardware,Internet Service...",1,1000000.0,1981-09-01,1975-04-04,10000+,fd80725f-53fc-7009-9878-aeecf1e9ffbb,company,organization
4,Compaq,"company,investor",USA,CA,SF Bay Area,Palo Alto,acquired,"Hardware,Information Technology,Software","Hardware,Information Technology,Software",1,1500000.0,1982-02-14,1982-02-14,11-50,10a3b2fd-b142-046b-7d8f-3b1aa4877aca,company,organization
6,Toyota Motor Corporation,"company,investor",JPN,NaN,NaN,NaN,ipo,"Automotive,Mobile,Transportation","Mobile,Transportation",1,42000000.0,1982-04-14,1937-08-28,10000+,12b90373-ab49-a56a-4b4e-c7b3e9236faf,company,organization


Once again we need to remove null values. This was another straightforward decision as this dataset is being used for our first research question. The first research question looks at the correlation between the location and industry of a company, if the company has no listed category then this would be impossible. We now only have 78,357 rows and 17 columns after removing the columns with no category. 

In [11]:
clean_notNull = clean_df1[~clean_df1["category_group_list"].isnull()]
clean_notNull.shape

(78357, 17)

To address the problem mentioned above regarding the category list seperated by commas:
* In order to allow us to analysis each industry in particular we seperated these industries into unique columns.
* This resolves the issue as we now don't have multiple industries being considered as a single industry in our analysis.

It is also important to note that there is quite a large number of extra columns after performing this step.

In [12]:
dummy_df1 = clean_notNull["category_group_list"].str.get_dummies(',')
dummy_df1.shape

(78357, 46)

It is important to have a look at our new dataframe with the industries seperated into seperate columns. It is interesting to note that if a company operates under a particular industry it is represented as a binary value in the table below. This is hugely beneficial for when it comes to analysis.

In [13]:
dummy_df1.head(5)

,Administrative Services,Advertising,Agriculture and Farming,Apps,Artificial Intelligence,Biotechnology,Clothing and Apparel,Commerce and Shopping,Community and Lifestyle,Consumer Electronics,Consumer Goods,Content and Publishing,Data and Analytics,Design,Education,Energy,Events,Financial Services,Food and Beverage,Gaming,Government and Military,Hardware,Health Care,Information Technology,Internet Services,Lending and Investments,Manufacturing,Media and Entertainment,Messaging and Telecommunications,Mobile,Music and Audio,Natural Resources,Navigation and Mapping,Payments,Platforms,Privacy and Security,Professional Services,Real Estate,Sales and Marketing,Science and Engineering,Software,Sports,Sustainability,Transportation,Travel and Tourism,Video
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [14]:
list(dummy_df1)

['Administrative Services',
 'Advertising',
 'Agriculture and Farming',
 'Apps',
 'Artificial Intelligence',
 'Biotechnology',
 'Clothing and Apparel',
 'Commerce and Shopping',
 'Community and Lifestyle',
 'Consumer Electronics',
 'Consumer Goods',
 'Content and Publishing',
 'Data and Analytics',
 'Design',
 'Education',
 'Energy',
 'Events',
 'Financial Services',
 'Food and Beverage',
 'Gaming',
 'Government and Military',
 'Hardware',
 'Health Care',
 'Information Technology',
 'Internet Services',
 'Lending and Investments',
 'Manufacturing',
 'Media and Entertainment',
 'Messaging and Telecommunications',
 'Mobile',
 'Music and Audio',
 'Natural Resources',
 'Navigation and Mapping',
 'Payments',
 'Platforms',
 'Privacy and Security',
 'Professional Services',
 'Real Estate',
 'Sales and Marketing',
 'Science and Engineering',
 'Software',
 'Sports',
 'Sustainability',
 'Transportation',
 'Travel and Tourism',
 'Video']

** Grouping of industry: **

It is quite convinent to use the cell above to copy a number of columns names if needed for code. Below we are grouping similar sub-industries into larger industry columns. The industries below were all very similar and fell under a more broad industry name. 

Technology in particular is one industry that stood out. Obviously a huge amount of technology companies exist in multiple different fields of technology today, but we thought it best to analyse this as one single industry. Similarly with finance and communication. 

In [15]:
Technology = ['Apps', 'Artificial Intelligence','Consumer Electronics','Data and Analytics','Gaming','Information Technology','Internet Services','Software']
Finance = ['Payments', 'Financial Services','Lending and Investments']
Communication = ['Messaging and Telecommunications','Mobile']

dummy_df1 = dummy_df1.assign(Technology=dummy_df1[Technology].max(1)).drop(Technology, 1)
dummy_df1 = dummy_df1.assign(Finance=dummy_df1[Finance].max(1)).drop(Finance, 1)
dummy_df1 = dummy_df1.assign(Communication=dummy_df1[Communication].max(1)).drop(Communication, 1)

dummy_df1.head(5)

,Administrative Services,Advertising,Agriculture and Farming,Biotechnology,Clothing and Apparel,Commerce and Shopping,Community and Lifestyle,Consumer Goods,Content and Publishing,Design,Education,Energy,Events,Food and Beverage,Government and Military,Hardware,Health Care,Manufacturing,Media and Entertainment,Music and Audio,Natural Resources,Navigation and Mapping,Platforms,Privacy and Security,Professional Services,Real Estate,Sales and Marketing,Science and Engineering,Sports,Sustainability,Transportation,Travel and Tourism,Video,Technology,Finance,Communication
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


** Merging ** 

Our next step is to merge our newly created industry type ('dummy_df') dataframe with our original dataframe ('clean_df1'). In order to do this we must reset the index of each dataframe, this enables use to merge the dataframes based on the index. This means that each column will be merged based on it's index number (e.g. Column 1 of clean_notNull => Column 1 of dummy_df1).

In [16]:
clean_notNull = clean_notNull.reset_index()
clean_notNull.head(5)

,index,company_name,roles,country_code,state_code,region,city,status,category_list,category_group_list,funding_rounds,funding_total_usd,last_funding_on,founded_on,employee_count,org_uuid,primary_role,type
0,0,Intel,"company,investor",USA,CA,SF Bay Area,Santa Clara,ipo,"Hardware,Manufacturing,Product Design,Semicond...","Design,Hardware,Manufacturing,Science and Engi...",1,2510000.0,1968-07-31,1968-07-18,10000+,1e4f199c-363b-451b-a164-f94571075ee5,company,organization
1,1,Intercomp,company,USA,OH,Cleveland,Medina,operating,"Hardware,Software","Hardware,Software",1,549000.0,1970-12-31,1968-01-01,101-250,6681b1b0-0cea-6a4a-820d-60b15793fa66,company,organization
2,3,Microsoft,"company,investor",USA,WA,Seattle,Redmond,ipo,"Cloud Computing,Collaboration,Consumer Electro...","Consumer Electronics,Hardware,Internet Service...",1,1000000.0,1981-09-01,1975-04-04,10000+,fd80725f-53fc-7009-9878-aeecf1e9ffbb,company,organization
3,4,Compaq,"company,investor",USA,CA,SF Bay Area,Palo Alto,acquired,"Hardware,Information Technology,Software","Hardware,Information Technology,Software",1,1500000.0,1982-02-14,1982-02-14,11-50,10a3b2fd-b142-046b-7d8f-3b1aa4877aca,company,organization
4,6,Toyota Motor Corporation,"company,investor",JPN,NaN,NaN,NaN,ipo,"Automotive,Mobile,Transportation","Mobile,Transportation",1,42000000.0,1982-04-14,1937-08-28,10000+,12b90373-ab49-a56a-4b4e-c7b3e9236faf,company,organization


In [17]:
dummy_df1 = dummy_df1.reset_index()
dummy_df1.head(5)

,index,Administrative Services,Advertising,Agriculture and Farming,Biotechnology,Clothing and Apparel,Commerce and Shopping,Community and Lifestyle,Consumer Goods,Content and Publishing,Design,Education,Energy,Events,Food and Beverage,Government and Military,Hardware,Health Care,Manufacturing,Media and Entertainment,Music and Audio,Natural Resources,Navigation and Mapping,Platforms,Privacy and Security,Professional Services,Real Estate,Sales and Marketing,Science and Engineering,Sports,Sustainability,Transportation,Travel and Tourism,Video,Technology,Finance,Communication
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


After we reset the indexes above, we need to merge the datasets. We are merging 'dummy_df1' to the right of the 'clean_notNull' dataframe. After merging it is no longer necessary to have the index, so we drop the index. It is good to print the first 5 columns of our new dataframe to ensure that all columns are accounted for. 

In [18]:
result_df1 = clean_notNull.merge(dummy_df1, on='index', how='right')
result_df1 = result_df1.drop(labels=['index'], axis=1)
result_df1.head(5)

,company_name,roles,country_code,state_code,region,city,status,category_list,category_group_list,funding_rounds,funding_total_usd,last_funding_on,founded_on,employee_count,org_uuid,primary_role,type,Administrative Services,Advertising,Agriculture and Farming,Biotechnology,Clothing and Apparel,Commerce and Shopping,Community and Lifestyle,Consumer Goods,Content and Publishing,Design,Education,Energy,Events,Food and Beverage,Government and Military,Hardware,Health Care,Manufacturing,Media and Entertainment,Music and Audio,Natural Resources,Navigation and Mapping,Platforms,Privacy and Security,Professional Services,Real Estate,Sales and Marketing,Science and Engineering,Sports,Sustainability,Transportation,Travel and Tourism,Video,Technology,Finance,Communication
0,Intel,"company,investor",USA,CA,SF Bay Area,Santa Clara,ipo,"Hardware,Manufacturing,Product Design,Semicond...","Design,Hardware,Manufacturing,Science and Engi...",1,2510000.0,1968-07-31,1968-07-18,10000+,1e4f199c-363b-451b-a164-f94571075ee5,company,organization,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Intercomp,company,USA,OH,Cleveland,Medina,operating,"Hardware,Software","Hardware,Software",1,549000.0,1970-12-31,1968-01-01,101-250,6681b1b0-0cea-6a4a-820d-60b15793fa66,company,organization,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,Microsoft,"company,investor",USA,WA,Seattle,Redmond,ipo,"Cloud Computing,Collaboration,Consumer Electro...","Consumer Electronics,Hardware,Internet Service...",1,1000000.0,1981-09-01,1975-04-04,10000+,fd80725f-53fc-7009-9878-aeecf1e9ffbb,company,organization,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Compaq,"company,investor",USA,CA,SF Bay Area,Palo Alto,acquired,"Hardware,Information Technology,Software","Hardware,Information Technology,Software",1,1500000.0,1982-02-14,1982-02-14,11-50,10a3b2fd-b142-046b-7d8f-3b1aa4877aca,company,organization,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Toyota Motor Corporation,"company,investor",JPN,NaN,NaN,NaN,ipo,"Automotive,Mobile,Transportation","Mobile,Transportation",1,42000000.0,1982-04-14,1937-08-28,10000+,12b90373-ab49-a56a-4b4e-c7b3e9236faf,company,organization,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


We have the same number of rows as each of our original dataframes, as well as 53 columns... having the same number of rows as each of the original dataframes is a good indicator that the merge was successful.

In [19]:
result_df1.shape

(78357, 53)

** Finally... **

The final step we wanted to take in this notebook before saving the dataset was to look at the countrys which make up the companies in our dataset. It is very important to understand the data you are dealing with for any form of data analysis. As you can see from below the USA makes up over half of our dataset. From seeing the below figures alone it is easy to choose a number of key countrys to analyse in particular.
* USA
* UK 
* IRL (Home country)

In [20]:
result_df1['country_code'].value_counts()

USA    44348
GBR     5788
CAN     2534
IND     2481
CHN     2144
FRA     1755
DEU     1380
ISR     1295
AUS     1062
ESP     1058
SWE      786
NLD      672
SGP      667
ITA      660
RUS      633
IRL      595
BRA      585
JPN      573
CHE      507
KOR      479
FIN      433
DNK      381
CHL      345
HKG      322
BEL      303
MEX      242
POL      233
ISL      231
ARG      224
TUR      214
       ...  
MOZ        2
SLE        2
DOM        2
DZA        2
MLI        2
BIH        2
HTI        2
SYC        2
HND        2
GRD        1
BRB        1
GGY        1
SOM        1
MNE        1
MDG        1
LBR        1
DMA        1
LAO        1
MTQ        1
MAC        1
BLZ        1
MAF        1
STP        1
UZB        1
COG        1
MRT        1
GUM        1
DJI        1
IRQ        1
AGO        1
Name: country_code, Length: 152, dtype: int64

### Saving resulting dataset from 200_prep_dataset1 in pickle

In [21]:
result_df1.to_pickle("../../data/processed/200_dataset1.pkl")